# Kapitel 8 aus dem Buch ...

Say you have a sequence of numbers, and you want to find its longest increasing subsequence - or one of them, if there are more.

In [1]:
from itertools import combinations

def naive_lis(seq):
    '''A Naive Solution to the Longest Increasing Subsequence Problem'''
    for length in range(len(seq), 0, -1):      # n, n-1, ..., 1
        for sub in combinations(seq, length):  # Subsequences of given length
            if list(sub) == sorted(sub):       # An increasing subsequence?
                return sub                     # Return it!

In [2]:
naive_lis([3,1,0,2,4])

(1, 2, 4)

### Don't Repeat Yourself
The basic idea of this chapter is to avoid (vermeiden) having your 'algorithm' repeat itself.
The principle is so simple, and even really easy to implement (at least (geringer) in Python), but the mojo (Glücksbringer) here is really deep (tiefgründig), as you'll see as we progress.

In [3]:
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)


from functools import wraps

def memo(func):
    '''A Memoizing Decorator. The idea of memoized function is that it caches its return values. If you call it a
    second time with the same parameters, it will simply return the cached value. You can certainly (allerdings) 
    put this sort of caching logic inside your function, but memo function is a more reusable (Mehrweg-) solution.
    '''
    cache = {}                           # Stored subproblem solutions
    @wraps(func)                         # Make wrap look like func
    def wrap(*args):                     # The memoized wrapper
        if args not in cache:            # Not already computed?
            cache[args] = func(*args)    # Compute & cache the solution
        return cache[args]               # Return the cached solution
    return wrap                          # Return the wrapper

fib = memo(fib)

In [4]:
fib(100)

573147844013817084101L

It's even designed to be used as a decorator:

In [5]:
@memo
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)

fib(100)

573147844013817084101L

- overlapping subproblems
- a recursion formulation of the powers of two.

### Option 1

In [6]:
@memo 
def two_pow(i):
    if i == 0: return 1
    return two_pow(i-1) + two_pow(i-1)

print(two_pow(10))
print(two_pow(100))

1024
1267650600228229401496703205376


overlapping subproblems
a recursion formulation of the powers of two.
### Option 2

In [7]:
def two_pow(i):
    if i == 0: return 1
    return 2 * two_pow(i-1)

print(two_pow(10))
print(two_pow(100))

1024
1267650600228229401496703205376


## Calculating binomial coefficients (Pascal's triangle)
We decompose the problem by conditioning (Aufbereitung) on whether some element is included.
$C(n,0) = 1$ for the single empty subset, and $C(0,k) = 0, k > 0$, for nonempty subsets of an empty set.
- path counting (abzählen)

In [8]:
@memo
def C(n,k):
    if k == 0: return 1
    if n == 0: return 0
    return C(n-1,k-1) + C(n-1,k)

print(C(4,2))
print(C(10,7))
print(C(100,50))

6
120
100891344545564193334812497256


In [9]:
from collections import defaultdict

n, k = 10, 7
C = defaultdict(int)
for row in range(n+1):
    C[row, 0] = 1
    for col in range(1,k+1):
        C[row,col] = C[row-1,col-1] + C[row-1,col]
        
C[n,k]

120

## Shortest Path in Directed Acyclic Graphs

not the classical way of expressing this alogirthm.
What is customarily done here, as in so many other DP algorithms, is to turn the algorithm "upside down" and make it iterative.

In [10]:
def celeb(G):
    n = len(G)
    u, v = 0, 1
    for c in range(2,n+1):
        if G[u][v]: u = c
        else:       v = c
    if u == n:      c = v
    else:           c = u
    for v in range(n):
        if c == v: continue
        if G[c][v]: break
        if not G[v][c]: break
    else:
        return c
    return None

In [11]:
from random import randrange
n = 100
G = [[randrange(2) for i in range(n)] for i in range(n)]
c = randrange(n)
for i in range(n):
    G[i][c] = True
    G[c][i] = False

In [12]:
print(celeb(G))

54


In [13]:
def naive_topsort(G, S=None):
    if S is None: S = set(G)                   # Default: All nodes
    if len(S) == 1: return list(S)             # Base case, single node
    v = S.pop()                                # Reduction: Remove a node
    seq = naive_topsort(G, S)                  # Recursion (assumption), n-1
    min_i = 0
    for i, u in enumerate(seq):
        if v in G[u]: min_i = i+1              # After all dependencies
    seq.insert(min_i, v)
    return seq

In [14]:
class bTree:
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
t = bTree(bTree("a", "b"), bTree("c", "d"))
t.right.left

'c'

In [15]:
class Tree:
    def __init__(self, kids, next=None):
        self.kids = self.val = kids
        self.next = next
        
t = Tree(Tree("a", Tree("b", Tree("c", Tree("d")))))
t.kids.next.next.val

'c'

In [16]:
def topsort(G):
    count = dict((u, 0) for u in G)            # The in-degree for each node
    for u in G:
        for v in G[u]:                         
            count[v] += 1                      # Count every in-edge
    Q = [u for u in G if count[u] == 0]        # Valid initial nodes
    S = []                                     # The result
    while Q:                                   # While we have start nodes...
        u = Q.pop()                            # Pick one
        S.append(u)                            # Use it as first of the rest
        for v in G[u]:
            count[v] -= 1                      # "Uncount" its out-edges
            if count[v] == 0:                  # New valid start nodes?
                Q.append(v)                    # Deal with them next
    return S

In [17]:
graph = { 1: [2,3], 2: [3], 3: [] } # QUELLE: https://pypi.python.org/pypi/digraphtools/0.1.0
topsort(graph)

[1, 2, 3]

In [18]:
graph_py = {'A': ['B', 'C'], # Quelle: https://www.python.org/doc/essays/graphs/
             'B': ['C', 'D'],
             'C': ['D'],
             'D': ['C'],
             'E': ['F'],
             'F': ['C']}
topsort(graph_py)

['E', 'F', 'A', 'B']

In [19]:
def find_shortest_path(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if not graph.has_key(start):
            return None
        shortest = None
        for node in graph[start]:
            if node not in path:
                newpath = find_shortest_path(graph, node, end, path)
                if newpath:
                    if not shortest or len(newpath) < len(shortest):
                        shortest = newpath
        return shortest

print(find_shortest_path(graph_py, "A", "D"))

['A', 'B', 'D']


## Gewichtet Graph

Gewicht: $w(u,v)$,
Distanz: $d(v)$,
Knoten (Node): $v, u$

In [20]:
DAG = { "a": ["b", "f"], "b": ["c", "d", "f"],
          "c": ["d"], "d": ["e", "f"],
          "e": ["f"],"f": [] }
DAG

{'a': ['b', 'f'],
 'b': ['c', 'd', 'f'],
 'c': ['d'],
 'd': ['e', 'f'],
 'e': ['f'],
 'f': []}

In [21]:
gDAG = {'a': {'b':2, 'f':9},
     'b': {'c':1, 'd':2, 'f':6},
     'c': {'d':7},
     'd': {'e':2, 'f':3},
     'e': {'f':4},
     'f': {}}
gDAG

{'a': {'b': 2, 'f': 9},
 'b': {'c': 1, 'd': 2, 'f': 6},
 'c': {'d': 7},
 'd': {'e': 2, 'f': 3},
 'e': {'f': 4},
 'f': {}}

In [22]:
def rec_dag_sp(W, s, t):                       # Shortest path from s to t
    '''Recursive, Memoized DAG Shortest Path'''
    @memo                                      # Memoize f
    def d(u):                                  # Distance from u to t
        if u == t: return 0                    # We're there!
        return min(W[u][v]+d(v) for v in W[u]) # Best of every first step
    return d(s)

In [23]:
def dag_sp(W, s, t):                           # Shortest path from s to t
    '''DAG Shortest Path'''
    d = {u:float('inf') for u in W}            # Distance estimates
    d[s] = 0                                   # Start node: Zero distance
    for u in topsort(W):                       # In top-sorted order...
        if u == t: break                       # Have we arrived?
        for v in W[u]:                         # For each out-edge...
            d[v] = min(d[v], d[u] + W[u][v])   # Relax the edge
    return d[t]                                # Distance to t (from s)

In [24]:
rec_dag_sp(gDAG, "a", "f")

7

In [25]:
dag_sp(gDAG, "a", "f")

7

The idea of the iterative algorithm is that as long as we have relaxed each edge out from each of your possible predecessors (that is, those earlier in topologically sorted order), we must necessarily (zwangsläufig) have relaxed all the in-edges (sich eindrängen) to you.

## Knapsack Strikes Back

we need to somehow define the subproblems, relate them to each other recursively, and then make sure we compute each subproblem only once (by using memoization, implicitly or explicitly).

- object $i$;
- weight $w[i]$;
- value $v[i]$;
- maximum value $m(r)$

$m(r)$ is the maximum value we can get with a (remaining) capacity $r$, each value of $r$ gives us a subproblem.
The recursiv decomposition is based in either (entweder) using or not using the last unit of the capacity.
If we don't use it, we have $m(r)=m(r-1)$.
If we do use it, we have to choose the right object to use.
If we choose (auswählen) object $i$ (provided (vorausgesetzt) it will fit in the remaining capacity), we would have $m(r)=v[i]+m(r-w[i])$, because we'd add the value of $i$, but we'd also have used up a portion of the remaining capacity equal to its weight.
We can choose whether or not to use the last capacity unit, and if we do use it, we can choose which object to add.

In [26]:
i = ["Diamant", "Schatzkarte", "Gold", "Rubin", "Holz", "Apple Watch"]
w = {i[0]: 2, i[1]: 1, i[2]: 4, i[3]: 2, i[4]: 3, i[5]: 1  }
v = {i[0]: 1, i[1]: 5, i[2]: 8, i[3]: 7, i[4]: 1, i[5]: 18 }
c = 10
print(i[0])
print(w)
print(v)

Diamant
{'Apple Watch': 1, 'Holz': 3, 'Gold': 4, 'Rubin': 2, 'Schatzkarte': 1, 'Diamant': 2}
{'Apple Watch': 18, 'Holz': 1, 'Gold': 8, 'Rubin': 7, 'Schatzkarte': 5, 'Diamant': 1}


In [27]:
def rec_unbounded_knapsack(w, v, c):            # Weights, values and capacity
    '''A Memoized Recursive Solution to the Unbounded Integer Knapsack Problem'''
    @memo                                       # m is memoized
    def m(r):                                   # Max val. w/remaining cap. r
        if r == 0: return "Irgendwie funkt es nicht"                     # No capacity? No value
        val = m(r-1)                            # Ignore the last cap. unit?
        for i, wi in enumerate(w):              # Try every object
            if wi > r: continue                 # Too heavy? Ignore it
            val = max(val, v[i] + m(r-wi))      # Add value, remove weight
        return val                              # Max over all last objects
    return m(c)                                 # Full capacity available

In [28]:
rec_unbounded_knapsack(w, v, c)

'Irgendwie funkt es nicht'

The running time here depends on the capacity and the number of objects.
Each memoized call $m(r)$ is computed only once, which means that for a capacity c, we have $O(c)$ calls.
Each call goes through all the $n$ objects, so the resulting running time is $O(cn)$.
Note that this is not a polynomial running time, because $c$ can grow exponentially with the actual problem size (the number of bits).
As mentioned earlier (genannt Abruf), this sort of running time is called "pseudopolynomial", and for reasonably (halbwegs) sized capacities, the solution is actually quite (ziemlich) efficient.

In [29]:
def unbounded_knapsack(w, v, c):
    '''An Iterative Solution to the Unbounded Integer Knapsack-Problem.
    As you can see, the two implementations are virtually identical,
    expect that the recursion is replaced with a for loop, and the
    cache is now a list.'''
    m = [0]  # Wieso spuckt er immer diesen Wert aus?
    for r in range(1,c+1):
        val = m[r-1]
        for i, wi in enumerate(w):
            if wi > r: continue
            val = max(val, v[i] + m[r-wi])
        m.append(val)
    return m[c]


In [30]:
unbounded_knapsack(w, v, c)

0

## 0-1 Knapsack Problem

Simular to the unbounded one. Defference is that we now add another parameter to the subproblem: in addition to restricting the capacity, we add the "in or out" idea and restrict how many of the objects we're allowed to use. $m(k,r)$ be the maximum value we can have with the first $k$ objects and a remaining capacity $r$.
$$k=0\oplus r=0 \widehat{=} m(i,r)=0$$

We only have to consider whether we want to include the last object $i=k-1$. If we don't, we will have $m(k,r)=m(k-1,r)$.
If the object is small enough, though, we can include it, meaning that $m(k,r)=v[i]+m(k-1,r.w[i])$, which is quite similar to the unbounded case, expect for the extra parameter $(k)$.

In [31]:
def rec_knapsack(w, v, c):                            # Weights, values and capacity
    '''A Memoized Recursiv Solution to the 0-1 Knapsack Problem'''
    @memo                                             # m is memoized
    def m(k, r):                                      # Max val., k objs and cap r
        if k == 0 or r == 0: return 0                 # No objects/no capacity
        i = k-1                                       # Object under consideration
        drop = m(k-1, r)                              # What if we drop the object?
        if w[i] > r: return drop                      # Too heavy: Must drop it
        return max(drop, v[i] + m(k-1, r-w[i]))       # Include it? Max of in/out
    return m(len(w), c)                               # All objects, all capacity

In [32]:
# rec_knapsack(w, v, c)

In [33]:
def knapsack(w, v, c):                                # Returns solution matrices
    '''An Iterative Solution to the 0-1 Knapsack Problem'''
    n = len(w)                                        # Number of available items
    m = [[0]*(c+1) for i in range(n+1)]               # Empty max-value matrix
    P = [[False]*(c+1) for i in range(n+1)]           # Empty keep/drop matrix
    for k in range(1, n+1):                           # We can use k first objects
        i = k-1                                       # Object under consideration
        for r in range(1,c+1):                        # Every positive capacity
            m[k][r] = drop = m[k-1][r]                # By default: drop the object
            if w[i] > r: continue                     # Too heavy? Ignore it
            keep = v[i] + m[k-1][r-w[i]]              # Value of keeping it
            m[k][r] = max(drop, keep)                 # Best of dropping and keeping
            P[k][r] = keep > drop                     # Did we keep it?
    return m, P                                       # Return full results

In [34]:
# knapsack(w, v, c)